## rag system for programers

In [4]:
import faiss
from sentence_transformers import SentenceTransformer

claim_no = "123"

# def get_claim_no(claim_no):
#         connecttion = db.connect(connect_url)
#         fetchdata= connecttion.get(claim_no)
        
model = SentenceTransformer('all-MiniLM-L6-v2')
docs = ["Paris is the capital of France.", 
        "Berlin is in Germany.",
        "Tokyo is in Japan.", 
        "Tokyo is capital of Japan.",
        "kapil is gen ai engineer working for testing model",
        "kapil works in gen Ai area",
        "Harshal is full stack engineer working for testing model",
        "kapil works in html and css area"
        ]

c:\Users\chetan chopade\Desktop\python_test_kapil\llm_test\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
docs_embedding = model.encode(docs)

dimension = docs_embedding.shape[1]

print(docs_embedding[7])


[-1.10196210e-01  1.20808696e-02 -3.78965633e-03  6.41784370e-02
 -1.53353589e-03 -3.80484723e-02 -3.45175788e-02 -1.13174461e-01
  8.92828852e-02  1.39240501e-02  1.48487920e-02  6.26398530e-03
 -9.64632817e-03  4.14922126e-02  5.89603521e-02 -1.38603244e-02
 -1.85832791e-02 -4.10539052e-03 -1.67046599e-02 -3.09441239e-02
 -1.43612167e-02 -8.29988793e-02  3.69534381e-02 -1.79431378e-03
  5.50072594e-03  4.44327369e-02 -1.20915221e-02  6.29442260e-02
  6.75253868e-02 -1.13190718e-01 -9.35795903e-03 -3.35168801e-02
  1.96087137e-02  2.26539932e-02 -3.31321806e-02  8.15760642e-02
 -9.29870605e-02  1.87006798e-02 -2.74472814e-02 -3.19294608e-03
  3.22279744e-02 -2.47253906e-02 -2.23012678e-02 -4.03832877e-03
  1.09560993e-02 -6.45299181e-02 -1.18226483e-01 -5.07696951e-03
 -3.26325782e-02 -1.93237979e-02 -8.95797461e-02 -4.16914113e-02
  6.74482584e-02 -1.09401755e-02 -4.80340645e-02  1.27990488e-02
  2.92264502e-02  4.89840060e-02  7.13990405e-02  8.59281048e-02
  3.77645083e-02 -2.57216

In [9]:
index = faiss.IndexFlatL2(dimension)
index.add(docs_embedding)

import pickle

# Step 2: Save index and docs
faiss.write_index(index, "faiss_index.index")
with open("docs.pkl", "wb") as f:
    pickle.dump(docs, f)

In [21]:
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# Load index and docs
index = faiss.read_index("faiss_index.index")
with open("docs.pkl", "rb") as f:
    docs = pickle.load(f)

# Load model and query
model = SentenceTransformer('all-MiniLM-L6-v2')
query = "what kapil does..?"
query_embedding = model.encode([query])

"""
docs = ["Paris is the capital of France.", 
    "Berlin is in Germany.",
    "Tokyo is in Japan.", 
    "Tokyo is capital of Japan.",
    "kapil is gen ai engineer working for testing model",
    "kapil works in gen Ai area",
    "Harshal is full stack engineer working for testing model",
    "kapil works in html and css area"
    ]
"""

# Search
top_k = 2
distances, indices = index.search(query_embedding, top_k)

# Show results
for i in indices[0]:
    print(docs[i])
    
    
"""
Hackers and cyber scammers are taking advantage of the coronavirus disease (COVID-19) pandemic by sending fraudulent email and WhatsApp messages that attempt to trick you into clicking on malicious links or opening attachments.

These actions can reveal your user name and password, which can be used to steal money or sensitive information.

If you are contacted by a person or organization that appears to be WHO, verify their authenticity before responding.

The World Health Organization will:

never ask for your username or password to access safety information
never email attachments you didn’t ask for
never charge money to apply for a job, register for a conference, or reserve a hotel
never conduct lotteries or offer prizes, grants, certificates or funding through email.
Be on alert when you receive an email with any link or attachments containing any reference to WHO. It might be a cyberattack.

The only call for donations WHO has issued is the COVID-19 Solidarity Response Fund, which is linked to below.  Any other appeal for funding or donations that appears to be from WHO is a scam.  

"""

kapil works in gen Ai area
kapil works in html and css area


In [ ]:
from typing import Dict, Any, List
from langchain.document_loaders import PyPDFLoader
from langchain.llms import OpenAI  # Or any other LLM you prefer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from os import getenv

def get_llm():
    llm = ChatOpenAI(
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    model_name="google/gemma-3-27b-it:free",
    )
    return llm


def summarize_claim_with_grounding(claim_details, supportive_pdf_content: str, llm) -> str:
    """
    Calls the LLM to summarize the claim document, grounded by the supportive PDF.
    """
    prompt_template = """You are an expert at summarizing claim documents.
    Here are the details of the claim:
    {claim_details}

    Use the following supportive document to ground your summary:
    ---
    {supportive_pdf_content}
    ---

    Summarize the key aspects of the claim, considering the information in both the claim details and the supportive document. Focus on providing a concise and informative summary relevant to the claim.
    """
    llm1 = ChatOpenAI(
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    model_name="google/gemma-3-27b-it:free",
    )
    
    prompt = PromptTemplate(template=prompt_template, input_variables=["claim_details", "supportive_pdf_content"])
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    summary = llm_chain.run(claim_details=claim_details, supportive_pdf_content=supportive_pdf_content)
    return summary

In [22]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from os import getenv

def create_pension_plan_table(employee_details: dict, pension_guidelines: str, llm) -> str:
    """
    Generates a pension plan summary in a tabular format based on employee details
    and pension guidelines.
    """
    prompt_template = """You are an expert at creating clear and user-friendly pension plan summaries in a tabular format.

    Here are the employee's details:
    {employee_details}

    Here are the guidelines of the pension plan:
    ---
    {pension_guidelines}
    ---

    Based on this information, create a personalized pension plan summary for the employee in a markdown table format. Include relevant details such as eligibility, contribution, vesting, estimated retirement benefits, etc., in a way that is easy to understand.
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["employee_details", "pension_guidelines"])
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    pension_plan = llm_chain.run(employee_details=employee_details, pension_guidelines=pension_guidelines)
    return pension_plan

# Example usage:
if __name__ == "__main__":
    supportive_pdf_content_pension = """
    Pension Plan Guidelines:

    Eligibility: Employees who have completed at least 5 years of service.
    Contribution Rate: Employee contributes 5% of their monthly salary, and the company matches 5%.
    Vesting Schedule:
    - After 5 years: 50% vested
    - After 7 years: 75% vested
    - After 10 years: 100% vested
    Retirement Age: 60 years.
    Investment Options: Employees can choose from three investment funds: Conservative, Moderate, and Aggressive.
    Withdrawal Rules: Full withdrawal is allowed upon retirement. Early withdrawals may incur penalties.
    """

    claim_details_pension = {
        "employee_name": "Kapil",
        "years_of_service": 8,
        "monthly_salary": 100000,
        "current_age": 35
    }

    # Initialize your LLM
    llm_pension = ChatOpenAI(
        openai_api_base=getenv("OPENROUTER_BASE_URL"),
        openai_api_key=getenv("OPENROUTER_API_KEY"),
        model_name="google/gemma-3-27b-it:free",
    )

    pension_plan_table = create_pension_plan_table(claim_details_pension, supportive_pdf_content_pension, llm_pension)
    print(pension_plan_table)

Okay, here's a personalized pension plan summary for Kapil, presented in a markdown table format, based on the provided employee details and pension plan guidelines.  I've aimed for clarity and user-friendliness.

```markdown
## Kapil's Pension Plan Summary

| **Section** | **Details** | **Explanation** |
|---|---|---|
| **Employee Information** |  |  |
| Name | Kapil |  |
| Current Age | 35 |  |
| Years of Service | 8 |  |
| Monthly Salary | ₹100,000 | Used for contribution calculations |
| **Eligibility** | Eligible | Kapil has completed 8 years of service, exceeding the 5-year minimum. |
| **Contributions** |  |  |
| Employee Contribution | ₹5,000 (5% of ₹100,000) | Automatically deducted from monthly salary. |
| Company Contribution | ₹5,000 (5% of ₹100,000) | Matched by the company. |
| Total Monthly Contribution | ₹10,000 | Combined employee and company contributions. |
| **Vesting Schedule** |  |  |
| Years of Service | Vesting Percentage |  |
| 5 Years | 50% |  |
| 7 Years | 75

## Kapil's Pension Plan Summary

| **Section** | **Details** | **Explanation** |
|---|---|---|
| **Employee Information** |  |  |
| Name | Kapil |  |
| Current Age | 35 |  |
| Years of Service | 8 |  |
| Monthly Salary | ₹100,000 | Used for contribution calculations |
| **Eligibility** | Eligible | Kapil has completed 8 years of service, exceeding the 5-year minimum. |
| **Contributions** |  |  |
| Employee Contribution | ₹5,000 (5% of ₹100,000) | Automatically deducted from monthly salary. |
| Company Contribution | ₹5,000 (5% of ₹100,000) | Matched by the company. |
| Total Monthly Contribution | ₹10,000 | Combined employee and company contributions. |
| **Vesting Schedule** |  |  |
| Years of Service | Vesting Percentage |  |
| 5 Years | 50% |  |
| 7 Years | 75% |  |
| 10 Years | 100% |  |
| **Current Vesting** | 75% | As of now, 75% of the accumulated pension fund is fully owned by Kapil. |
| **Retirement Details** |  |  |
| Retirement Age | 60 Years |  |